## Text Generation using LSTMs

Text Generation is a type of Language Modelling problem. Language Modelling is the core problem for a number of of natural language processing tasks such as speech to text, conversational system, and text summarization. A trained language model learns the likelihood of occurrence of a word based on the previous sequence of words used in the text. Language models can be operated at character level, n-gram level, sentence level or even paragraph level. In this notebook, I created a language model for generating natural language text by implement and training state-of-the-art Recurrent Neural Network.

In [2]:
# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
curr_dir = r'C:\Users\balajiam\Documents\ML Data Analysis\New York Times Comments\\'
all_headlines = []
for filename in os.listdir(curr_dir):
    if 'Articles' in filename:
        article_df = pd.read_csv(curr_dir + filename)
        all_headlines.extend(list(article_df.headline.values))
        break

all_headlines = [h for h in all_headlines if h != "Unknown"]
len(all_headlines)

831

In [6]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 

corpus = [clean_text(x) for x in all_headlines]
corpus[:10]

['finding an expansive view  of a forgotten people in niger',
 'and now  the dreaded trump curse',
 'venezuelas descent into dictatorship',
 'stain permeates basketball blue blood',
 'taking things for granted',
 'the caged beast awakens',
 'an everunfolding story',
 'oreilly thrives as settlements add up',
 'mouse infestation',
 'divide in gop now threatens trump tax plan']

In [7]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:10]

[[169, 17],
 [169, 17, 665],
 [169, 17, 665, 367],
 [169, 17, 665, 367, 4],
 [169, 17, 665, 367, 4, 2],
 [169, 17, 665, 367, 4, 2, 666],
 [169, 17, 665, 367, 4, 2, 666, 170],
 [169, 17, 665, 367, 4, 2, 666, 170, 5],
 [169, 17, 665, 367, 4, 2, 666, 170, 5, 667],
 [6, 80]]

In [8]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [9]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 18, 10)            24220     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2422)              244622    
Total params: 313,242
Trainable params: 313,242
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(predictors, label, epochs=100)

Epoch 1/100
4806/4806 [==============================] - 9s 2ms/step - loss: 7.3791
Epoch 2/100
4806/4806 [==============================] - 4s 752us/step - loss: 6.9077
Epoch 3/100
4806/4806 [==============================] - 3s 690us/step - loss: 6.7909
Epoch 4/100
4806/4806 [==============================] - 4s 774us/step - loss: 6.7115
Epoch 5/100
4806/4806 [==============================] - 5s 967us/step - loss: 6.6333
Epoch 6/100
4806/4806 [==============================] - 4s 915us/step - loss: 6.5438
Epoch 7/100
4806/4806 [==============================] - 5s 1ms/step - loss: 6.4428
Epoch 8/100
4806/4806 [==============================] - 5s 943us/step - loss: 6.3313
Epoch 9/100
4806/4806 [==============================] - 4s 845us/step - loss: 6.2189
Epoch 10/100
4806/4806 [==============================] - 5s 1ms/step - loss: 6.1045
Epoch 11/100
4806/4806 [==============================] - 4s 907us/step - loss: 5.9928
Epoch 12/100
4806/4806 [==============================] - 

4806/4806 [==============================] - 8s 2ms/step - loss: 1.7535
Epoch 95/100
4806/4806 [==============================] - 7s 2ms/step - loss: 1.7343
Epoch 96/100
4806/4806 [==============================] - 8s 2ms/step - loss: 1.7169
Epoch 97/100
4806/4806 [==============================] - 8s 2ms/step - loss: 1.6993
Epoch 98/100
4806/4806 [==============================] - 9s 2ms/step - loss: 1.6721
Epoch 99/100
4806/4806 [==============================] - 9s 2ms/step - loss: 1.6527
Epoch 100/100
4806/4806 [==============================] - 9s 2ms/step - loss: 1.6327A


In [11]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [13]:
print (generate_text("united states", 15, model, max_sequence_len))
print (generate_text("preident trump", 41, model, max_sequence_len))
print (generate_text("donald trump", 4, model, max_sequence_len))
print (generate_text("india and china", 14, model, max_sequence_len))
print (generate_text("new york", 10, model, max_sequence_len))
print (generate_text("science and technology", 16, model, max_sequence_len))

United States York Today A Holocaust Survivors Story On Stage Trump Trump Them Them Say Pick An
Preident Trump Mailbag A Patriots Photo Lets Trump Score A Point Says By The Times Does Too Elegy An Scout By Too Soldier Thats Concerns Say Safety Concerns Too Intervened An Soldier Will Rages Transgender Risk Everunfolding Elegy Chelsea Tried Chelsea Transgender The
Donald Trump May Helping Plan And
India And China You Old Divide To Avoid Chaos Make Black Toddlers For Them All Them Do
New York Today A Holocaust Survivors Story On Stage This He Medicine
Science And Technology Going Yanks Steal Mets Thunder Over Banning A Border Of Fathers Day On Oranges To Pick
